In [22]:
from linkers_and_taggers import wikifier_tag, stanford_tag, stanford_tagger
from sample_data import n_samples
from json_extractor import from_file_get_n_docs

In [24]:
raw_docs = n_samples(10)
stan_docs = from_file_get_n_docs('50_tagged_by_stanford.jsonl', 10)
wiki_docs = from_file_get_n_docs('100_tagged_by_wikifier.jsonl', 10)
len(raw_docs), len(stan_docs), len(wiki_docs)

(10, 10, 10)

In [33]:
test_doc_index = 2

In [34]:
describe_token_array = lambda toks, name=None: (u'{0} {1} tokens'.format(len(toks), name), u'|| '.join(toks))

# Wikifier tokens observations
    1. No punctuation as word tokens. This only means that the surface forms only include words and no punctuation. It makes sense for a surface_form to not be split by punctuation. When I want to index stanford's words, I have to skip punctuation. However, I have to keep punctuation for stanford to tag. So, maybe I can make a dictionary that maps the token index in stanford, to the word index.
    2. ' are kept as part of the token for It's, I don't know about possesives.

In [35]:
wiki_tokens = wiki_docs[test_doc_index]['words']
describe_token_array(wiki_tokens, 'wiki')

(u'118 wiki tokens',
 u'Home|| Style|| The|| Return|| Of|| The|| Nike|| Air|| Max|| Sensation|| Has|| 80|| s|| Babies|| Hyped|| Posted|| on|| Sep|| 22|| 2015|| If|| you|| were|| a|| basketball|| fan|| who|| was|| born|| in|| the|| 80s|| you|| were|| lucky|| enough|| to|| witness|| the|| beauty|| that|| is|| 90s|| basketball|| It|| was|| truly|| a|| great|| time|| to|| be|| basketball|| fan|| If|| you|| played|| close|| attention|| to|| what|| the|| players|| were|| wearing|| on|| their|| feet|| you|| would|| have|| also|| noticed|| the|| wide|| array|| of|| footwear|| these|| player|| used|| to|| rock|| One|| of|| those|| happens|| to|| the|| the|| Nike|| Air|| Max|| Sensation|| which|| is|| also|| set|| to|| receive|| the|| retro|| treatment|| this|| year|| Originally|| released|| back|| in|| read|| more|| Author|| KicksOnFire|| Share|| This|| Post|| On|| GoogleFacebookTwitter')

# Current Stanford tokenization scheme I use
    1. In doc index no. 2 it keeps the >> (\xbb) weird chars (with proper encoding of course)
    2. In doc index no. 2 it keeps the ... (\u2026) weird chars

In [41]:
stan_tokens = [t[0] for t in stan_docs[test_doc_index]]
describe_token_array(stan_tokens, 'stanford')
# print describe_token_array(stan_tokens, 'stanford')[1]

(u'119 stanford tokens',
 u'Home|| \xbb|| Style|| \xbb|| The|| Return|| Of|| The|| Nike|| Air|| Max|| Sensation|| Has|| 80\u2019s|| Babies|| Hyped!|| Posted|| on|| Sep|| 22,|| 2015|| If|| you|| were|| a|| basketball|| fan|| who|| was|| born|| in|| the|| 80s,|| you|| were|| lucky|| enough|| to|| witness|| the|| beauty|| that|| is|| 90s|| basketball.|| It|| was|| truly|| a|| great|| time|| to|| be|| basketball|| fan.|| If|| you|| played|| close|| attention|| to|| what|| the|| players|| were|| wearing|| on|| their|| feet|| you|| would|| have|| also|| noticed|| the|| wide|| array|| of|| footwear|| these|| player|| used|| to|| rock.|| One|| of|| those|| happens|| to|| the|| the|| Nike|| Air|| Max|| Sensation,|| which|| is|| also|| set|| to|| receive|| the|| retro|| treatment|| this|| year!|| Originally|| released|| back|| in|| \u2026read|| more|| Author:|| KicksOnFire|| Share|| This|| Post|| On|| GoogleFacebookTwitter')

# I have two options for dealing with differences due to special characters in text.
    1. Pre process the text for stanford to remove all special characters.
    2. Keep the special characters and build a function that looks for similar words in a token range.

## Option no. 2:  Keep the special chars, build a func that looks for similar words in range.